In [ ]:
import json
import os
import datetime
import logging
import pandas as pd
from typing import Dict, List
from dataclasses import dataclass
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from autogen_ext.models.anthropic import AnthropicBedrockChatCompletionClient, BedrockInfo
from autogen_core.models import ModelInfo

# === Data Class for Configuration ===      # PASS THE CONFIGURATION
@dataclass
class CampaignInstruction:
    natural_language_prompt: str
    email_template: str
    metadat : str

# === Main Data-Aware Email Campaign Class ===
class DataDrivenEmailCampaign:
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df = pd.read_csv(csv_path)
        self.schema = self.df.dtypes.to_dict()
        self.sample_rows = self.df.head(3).to_dict(orient='records')
        self.setup_logging()
        self.setup_aws_client()
        self.setup_agents()

    def setup_logging(self):
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)

    def setup_aws_client(self):
        self.bedrock_info = BedrockInfo(
            aws_access_key=os.getenv("AWS_ACCESS_KEY"),
            aws_secret_key=os.getenv("AWS_SECRET_KEY"),
            aws_session_token=os.getenv("AWS_SESSION_TOKEN"),
            aws_region=os.getenv("AWS_REGION", "us-west-2")
        )
        self.llm_client = AnthropicBedrockChatCompletionClient(
            model="anthropic.claude-3-sonnet-20240229-v1:0",
            model_info=ModelInfo(
                vision=False,
                function_calling=True,
                json_output=True,
                family="claude",
                structured_output=True
            ),
            bedrock_info=self.bedrock_info
        )
        self.llm_config = {"client": self.llm_client}

    def setup_agents(self):
        self.sql_agent = AssistantAgent(
            name="SQLGenerator",
            system_message="""
            You are a SQL expert. Based on the provided schema and instruction, generate a SELECT SQL query.
            Output JSON: {"query": "SELECT ... WHERE ...;"}                                                     
            """,
            llm_config=self.llm_config,
        )

        self.email_agent = AssistantAgent(
            name="EmailCreator",
            system_message="""
            You are an email copywriting assistant. Based on filtered user data and a prompt, write personalized emails.
            Output JSON: List of {"name": ..., "email": ..., "subject": ..., "body": ...}
            """,
            llm_config=self.llm_config,
        )

        self.compliance_agent = AssistantAgent(
            name="ComplianceChecker",
            system_message="""
            You are a compliance checker. Review email content and validate if it meets GDPR and CAN-SPAM standards.
            Output JSON: {"gdpr": true/false, "can_spam": true/false, "notes": "..."}
            """,
            llm_config=self.llm_config,
        )

        self.performance_agent = AssistantAgent(
            name="PerformanceAdvisor",
            system_message="""
            You are a campaign performance advisor. Based on the campaign objective and user segments, suggest:
            - Optimal send time
            - A/B testing opportunities
            - Subject line improvements
            Output JSON: {"send_time": ..., "ab_tests": [...], "subject_suggestions": [...]}
            """,
            llm_config=self.llm_config,
        )

        self.insight_agent = AssistantAgent(
            name="SegmentInsights",
            system_message="""
            You are a segmentation analyst. Based on sample user data and schema, identify user segments:
            - Dormant, Active, VIP, New
            Output JSON: List of {"user_id": ..., "segment": ..., "reason": ...}
            """,
            llm_config=self.llm_config,
        )

        self.user_proxy = UserProxyAgent(
            name="CampaignManager",
            human_input_mode="NEVER",
            code_execution_config={"work_dir": "campaign_data"},
            max_consecutive_auto_reply=3,
            is_termination_msg=lambda x: "EMAIL_CAMPAIGN_DONE" in x.get("content", "")
        )

        self.group_chat = GroupChat(
            agents=[
                self.user_proxy,
                self.sql_agent,
                self.insight_agent,
                self.email_agent,
                self.compliance_agent,
                self.performance_agent
            ],
            messages=[],
            max_round=15
        )
        self.manager = GroupChatManager(groupchat=self.group_chat, llm_config=self.llm_config)

    def execute(self, instruction: CampaignInstruction):
        prompt = f"""
        SCHEMA: {json.dumps({k: str(v) for k, v in self.schema.items()})}
        SAMPLE ROWS: {json.dumps(self.sample_rows)}

        TASK: {instruction.natural_language_prompt}

        METADATA : {instruction.metadata}

        EMAIL TEMPLATE: {instruction.email_template}

        Please generate the SQL query, segment users, and then create compliant, personalized emails.
        EMAIL_CAMPAIGN_DONE
        """
        self.user_proxy.initiate_chat(self.manager, message=prompt)
        self.save_transcript()

    def save_transcript(self):
        os.makedirs("campaign_data", exist_ok=True)
        ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        with open(f"campaign_data/transcript_{ts}.json", "w") as f:
            json.dump(self.group_chat.messages, f, indent=2)
        self.logger.info("Saved group chat transcript.")

# === Example Execution ===

def run():
    campaign = DataDrivenEmailCampaign("your_users.csv")

    instruction = CampaignInstruction(
        natural_language_prompt="Select Premium users inactive for more than 90 days.",     ## pass the naturel language promt whaterver we want
        metadata = f"{"pass the column names over here"}"
        email_template="""
        Hi {name},\n\nWe noticed you haven't logged in for a while. Explore new features in your {account_type} account.\n\nLog back in today →\n        """
    )

    campaign.execute(instruction)

if __name__ == "__main__":
    run()

In [ ]:
llm_config={
    "config_list": [
        {
            "model": "anthropic.claude-3-5-sonnet-20241022-v2:0",
            "api_type": "bedrock",
            "aws_access_key": creds.access_key,
            "aws_secret_key": creds.secret_key,
            "aws_region": "us-west-2",
        }
    ],
    "temperature": 0.7,
    "max_tokens": 1000
}


## SQL QUERY GENERTION BASED ON THE METADATA

In [1]:
# import os
# import logging
# import boto3
# from langchain_aws import BedrockEmbeddings, ChatBedrock
# from langchain.text_splitter import CharacterTextSplitter
# from langchain_community.vectorstores import FAISS
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_community.document_loaders.csv_loader import CSVLoader
# import json
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_community.document_loaders import PyMuPDFLoader, CSVLoader
# import altair as alt
# import pandas as pd
# import vl_convert
# from langchain_core.documents import Document
# from dotenv import load_dotenv

# # Load environment variables
# load_dotenv()


# # Configure logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# # Use temporary credentials directly
# assumed_role_session = boto3.Session(
#     aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
#     aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
#     aws_session_token=os.getenv("AWS_SESSION_TOKEN")
# )

# bedrock_client = assumed_role_session.client("bedrock-runtime")
# bedrock_embeddings = BedrockEmbeddings(
#     model_id=os.getenv("EMBED_MODEL_ID"),
#     client=bedrock_client
# )

# class TextToSQL:
#     def __init__(self, csv_path: str, pdf_path: str, faiss_index_path: str):
#         self.csv_path = csv_path
#         self.pdf_path = pdf_path
#         self.faiss_index_path = faiss_index_path
#         self.embeddings = bedrock_embeddings
#         self.vectorstore = None
#         self.docs = []
#         self.load_csv_documents()
#         self.load_pdf_documents()
#         self.create_vectorstore()
#         self.load_vectorstore()
#         self.session = boto3.Session(profile_name="test")
#         self.client = self.session.client("bedrock-runtime")

#     def load_csv_documents(self):
#         """Loads CSV data and adds metadata."""
#         if self.csv_path:
#             # loader = CSVLoader(file_path=self.csv_path, csv_args={"delimiter": ",", "quotechar": '"'})
#             # documents = loader.load()
#             df = pd.read_csv(self.csv_path, delimiter=",", quotechar='"')
    
#             # Create Document objects from DataFrame rows
#             documents = []
            
#             # Option 1: If you want to use a specific column as content
#             # Replace 'text_column' with your actual column name containing the text
#             for i, row in df.iterrows():
#                 content = str(row['text_column'])  # Replace with your column name
#                 metadata = {'source': self.csv_path, 'row': i}
#                 documents.append(Document(page_content=content, metadata=metadata))
            
#             text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
#             split_docs = text_splitter.split_documents(documents)
            
#             for doc in split_docs:
#                 doc.metadata["source"] = "csv"
            
#             self.docs.extend(split_docs)

#     def load_pdf_documents(self):
#         """Loads PDF data and adds metadata (NO OCR REQUIRED)."""
#         if self.pdf_path:
#             loader = PyMuPDFLoader(self.pdf_path)  # No OCR required
#             documents = loader.load()
#             text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
#             split_docs = text_splitter.split_documents(documents)
            
#             for doc in split_docs:
#                 doc.metadata["source"] = "pdf"
            
#             self.docs.extend(split_docs)

#     def create_vectorstores(self):
#         """Creates and saves the FAISS vector store."""
#         try:
#             sample_text = self.docs[0].page_content
#             sample_embedding = self.generate_embedding(sample_text)
            
#             if sample_embedding is None or len(sample_embedding) == 0:
#                 logger.error("Embeddings are not generated correctly.")
#                 raise ValueError("Embeddings are not generated correctly.")
#         except Exception as e:
#             logger.error(f"Error generating embeddings: {e}")
#             raise

#         self.vectorstore = FAISS.from_documents(self.docs, self.embeddings)
#         self.vectorstore.save_local(self.faiss_index_path)


#     def create_vectorstore(self):
#         """Loads an existing FAISS index if available; otherwise, creates and saves a new one."""
#         try:
#             # Check if FAISS index exists
#             if os.path.exists(self.faiss_index_path):
#                 logger.info(f"FAISS index found at {self.faiss_index_path}, loading it...")
#                 self.vectorstore = FAISS.load_local(
#                     self.faiss_index_path, self.embeddings, allow_dangerous_deserialization=True
#                 )
#                 logger.info("FAISS index loaded successfully.")
#                 return  # Correctly returning after loading

#             if not self.docs:
#                 logger.error("Document list is empty. Cannot create vector store.")
#                 raise ValueError("Document list is empty.")

#             sample_text = self.docs[0].page_content
#             sample_embedding = self.generate_embedding(sample_text)

#             if not sample_embedding or len(sample_embedding) == 0:
#                 logger.error("Embeddings are not generated correctly.")
#                 raise ValueError("Embeddings are not generated correctly.")

#             if not hasattr(self, 'embeddings') or self.embeddings is None:
#                 logger.error("Embeddings function is not defined.")
#                 raise ValueError("Embeddings function is not defined.")

#             # Create and save FAISS vector store
#             logger.info("No existing FAISS index found. Creating a new one...")
#             self.vectorstore = FAISS.from_documents(self.docs, self.embeddings)
#             self.vectorstore.save_local(self.faiss_index_path)
#             logger.info(f"FAISS vector store successfully saved at {self.faiss_index_path}")

#         except Exception as e:
#             logger.error(f"Error while handling FAISS index: {e}")
#             raise


#     def generate_embedding(self, text):
#         """Generates embeddings for the given text."""
#         return self.embeddings.embed_query(text)

#     def load_vectorstore(self):
#         """Loads the FAISS vector store from disk."""
#         self.vectorstore = FAISS.load_local(self.faiss_index_path, self.embeddings, allow_dangerous_deserialization=True)

#     def retrieve_context(self, query: str, k: int = 10):
#         """Retrieves relevant context from both CSV (for SQL) and PDF (for explanations)."""
        
#         retrieved_docs = self.vectorstore.similarity_search(query, k=k)
        
#         csv_content = "\n".join([doc.page_content for doc in retrieved_docs if doc.metadata.get("source") == "csv"])
#         pdf_content = "\n".join([doc.page_content for doc in retrieved_docs if doc.metadata.get("source") == "pdf"])

#         return pdf_content, csv_content
    
    

#     def save_pie_chart(self,data, filename="dynamic_pie_chart.png", inner_radius=50):
#         """
#         Generates a pie chart and saves it as PNG.

#         Parameters:
#             data (list of dict): Data format [{"category": value, "count": value}]
#             filename (str): Output PNG file name.
#             inner_radius (int): Set to 0 for full pie, default 50 for donut chart.
#         """
#         df = pd.DataFrame(data["Visualization"]["data"]["values"])  # Directly use data

#         chart = alt.Chart(df).mark_arc(innerRadius=inner_radius).encode(
#             theta=alt.Theta("count:Q"),
#             color=alt.Color(df.columns[0] + ":N")  # Auto-detect category
#         )

#         png_bytes = vl_convert.vegalite_to_png(chart.to_json())
#         with open(filename, "wb") as f:
#             f.write(png_bytes)

#         print(f"Pie chart saved as '{filename}'")
    
#     def generate_sql_query(self, query: str):
#         """Generates an SQL query based on the retrieved context and user query."""
#         pdf_context, csv_context = self.retrieve_context(query)
#         json_output ={
#             "Answer": "To analyze the gender distribution across the data, we can generate a count of students by gender and visualize it in a bar chart.",
#             "SQL Query": "SELECT gender, COUNT(*) as count FROM students GROUP BY gender",
#             "SQL Query Answer": "male, 8\nfemale, 4",
#             "Visualization": {
#                 "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
#                 "description": "Gender Distribution",
#                 "data": {
#                 "values": [
#                     {"gender": "male", "count": 8},
#                     {"gender": "female", "count": 4}
#                 ]
#                 },
#                 "mark": "bar",
#                 "encoding": {
#                 "x": {"field": "gender", "type": "nominal"},
#                 "y": {"field": "count", "type": "quantitative"},
#                 "color": {"field": "gender", "type": "nominal"}
#         }
#         }
#         }       # Convert json_output to a JSON string
#         json_output_str = json.dumps(json_output, indent=2)
#         json_output_str = json_output_str.replace("{", "{{").replace("}", "}}")

#         # Create the prompt for Claude 3.5 Sonnet
#         prompt = {
#             "anthropic_version": "bedrock-2023-05-31",
#             "max_tokens": 2048,
#             "system": """You are an expert AI assistant for Text-to-SQL conversion.

#             **Task:**
#             - Analyze the provided PDF for context and the CSV data structure to understand the schema and answer the user's question:
#             1. If it requires data retrieval, generate a SQL query and summarize the answer using the PDF context.
#             2. If it doesn't require SQL querying, provide only an explanation and set the SQL query to null.
#             3. If the question involves time-based data or any data that would benefit from visualization (especially date ranges, timestamps, or categorical distributions), also generate a Vega-Lite specification.""",
            
            
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": f"""Explanation:\n{pdf_context}\n\nData Structure:\n{csv_context}\n\nQuestion: {query}\n\n**Instructions:**

#             Format your response ONLY as a valid JSON object with these fields:
#             - Answer: Your explanation based on the PDF context
#             - SQL Query: The SQL query to retrieve the requested data, or null if no query is needed
#             - SQL Query Answer: The result of the SQL query in a simple format
#             - Visualization: When appropriate, include a Vega-Lite JSON specification for visualization

#             For Vega-Lite visualizations:
#             - Use appropriate chart types based on the data (bar charts for categorical data, line charts for time series, etc.)
#             - Keep the specification clean and minimal
#             - For categorical distributions, use bar charts or pie charts as appropriate
#             - IMPORTANT: Do not add any text before or after the JSON object. Your entire response should be only the JSON object itself.

#             Example of exactly how your response should look:
#             {json_output_str}"""
            
#                 }
#             ],
#             "temperature": 0.0
#         }

#         # Convert prompt to JSON string
#         input_text = json.dumps(prompt)
        
#         # Call Claude 3.5 Sonnet model
#         response = self.client.invoke_model(
#             modelId=os.getenv("MODEL_ID"), 
#             body=input_text
#         )

#         # Parse the response
#         response_body = json.loads(response['body'].read().decode("utf-8"))
#         sql_query = response_body['content'][0]['text']
#         out = json.loads(sql_query)
#         try:
#             self.save_pie_chart(out)
#         except:
#             pass

       
#         return sql_query
    
#     def generate_sql_query_with_stream(self, query: str):
#         """Generates an SQL query based on the retrieved context and user query with streaming."""
#         pdf_context, csv_context = self.retrieve_context(query)
#         json_output = {
#             "Answer": "To analyze the gender distribution across the data, we can generate a count of students by gender and visualize it in a bar chart.",
#             "SQL Query": "SELECT gender, COUNT(*) as count FROM students GROUP BY gender",
#             "SQL Query Answer": "male, 8\nfemale, 4",
#             "Visualization": {
#                 "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
#                 "description": "Gender Distribution",
#                 "data": {
#                 "values": [
#                     {"gender": "male", "count": 8},
#                     {"gender": "female", "count": 4}
#                 ]
#                 },
#                 "mark": "bar",
#                 "encoding": {
#                 "x": {"field": "gender", "type": "nominal"},
#                 "y": {"field": "count", "type": "quantitative"},
#                 "color": {"field": "gender", "type": "nominal"}
#                 }
#             }
#         }
        
#         # Convert json_output to a JSON string
#         json_output_str = json.dumps(json_output, indent=2)
#         json_output_str = json_output_str.replace("{", "{{").replace("}", "}}")

#         # Create the prompt for Claude 3.5 Sonnet
#         prompt = {
#             "anthropic_version": "bedrock-2023-05-31",
#             "max_tokens": 2048,
#             "system": """You are an expert AI assistant for Text-to-SQL conversion.

#             **Task:**
#             - Analyze the provided PDF for context and the CSV data structure to understand the schema and answer the user's question:
#             1. If it requires data retrieval, generate a SQL query and summarize the answer using the PDF context.
#             2. If it doesn't require SQL querying, provide only an explanation and set the SQL query to null.
#             3. If the question involves time-based data or any data that would benefit from visualization (especially date ranges, timestamps, or categorical distributions), also generate a Vega-Lite specification.""",
            
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": f"""Explanation:\n{pdf_context}\n\nData Structure:\n{csv_context}\n\nQuestion: {query}\n\n**Instructions:**

#             Format your response ONLY as a valid JSON object with these fields:
#             - Answer: Your explanation based on the PDF context
#             - SQL Query: The SQL query to retrieve the requested data, or null if no query is needed
#             - SQL Query Answer: The result of the SQL query in a simple format
#             - Visualization: When appropriate, include a Vega-Lite JSON specification for visualization

#             For Vega-Lite visualizations:
#             - Use appropriate chart types based on the data (bar charts for categorical data, line charts for time series, etc.)
#             - Keep the specification clean and minimal
#             - For categorical distributions, use bar charts or pie charts as appropriate
#             - IMPORTANT: Do not add any text before or after the JSON object. Your entire response should be only the JSON object itself.

#             Example of exactly how your response should look:
#             {json_output_str}"""
#                 }
#             ],
#             "temperature": 0.0
#         }

#         # Convert prompt to JSON string
#         input_text = json.dumps(prompt)
        
#         # Call Claude 3.5 Sonnet model with streaming
#         response_stream = self.client.invoke_model_with_response_stream(
#             modelId=os.getenv("MODEL_ID"),
#             body=input_text
#         )
        
#         # Process the streaming response
#         complete_response = ""
        
#         for event in response_stream["body"]:
#             if "chunk" in event:
#                 chunk_data = json.loads(event["chunk"]["bytes"].decode("utf-8"))
#                 if "content" in chunk_data and len(chunk_data["content"]) > 0:
#                     text_chunk = chunk_data["content"][0].get("text", "")
#                     complete_response += text_chunk
#                     # We yield each chunk as it arrives
#                     yield text_chunk, complete_response
        
#         # Try to parse the final response and save pie chart
#         try:
#             final_out = json.loads(complete_response)
#             try:
#                 self.save_pie_chart(final_out)
#             except:
#                 pass
#         except json.JSONDecodeError:
#             pass
    



# if __name__ == "__main__":
#     text2sql = TextToSQL(csv_path="data/data.csv", pdf_path="data/data.pdf", faiss_index_path="faiss_index")
#     query = "Fetch the GPA of id number 1141"
#     # query = "What is ID and Class"
#     sql_query = text2sql.generate_sql_query(query)
#     print(sql_query)

In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("/home/sarveshharikant/EXIMIETAS/git_AWS/aws/misc/dataset/data.csv")
column_names = df.columns.astype(str).tolist()

print("list of column names",column_names)

## THE SAME WAY YOU CAN CHANGE THE PROMPT IN YOUR INVOKING CODE

In [ ]:
import json
import boto3

MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"

bedrock_runtime = boto3.client("bedrock-runtime")

system_prompt = '''
You are a specialized SQL query generation agent. Your task is to create simple SQL queries based on user requests and provided column names.

Output format must be strictly in JSON:
{"SQL": "SELECT ID, gender\nFROM table_name"}

Generate only the JSON output without any additional information or explanations.
'''

user_prompt = f"{column_names} fetch me the id and gender"    ## PASS THE COLUMNS NAME IN THSI column_names

messages = [
    {
        "role": "user",
        "content": [
            {
                "text": user_prompt
            },
        ],
    }
]

response = bedrock_runtime.converse(
    system = [
        {
          "text": system_prompt
        }
    ],
    modelId = MODEL_ID,
    messages = messages
)
response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

In [5]:
import json

response = json.loads(response_text)

In [ ]:
response['SQL']

In [1]:
import pandas as pd
ans = {"columns": ['banker', 'oppuruting'], 'data': [['jame',1], ['andrew', 2],['jame',1], ['andrew', 2]]}

In [2]:
df = pd.DataFrame(ans['data'], columns=ans['columns'])


In [ ]:
df.head()

In [ ]:
import os
import chainlit as cl
from aws_rag_pdf import TextToSQL
import json
import time  # For simulating streaming delay

# Global variables to store uploaded file paths
csv_path = None
pdf_path = None
text2sql = None  

@cl.on_message
async def handle_message(message: cl.Message):
    global csv_path, pdf_path, text2sql

    # Check if files are uploaded
    if message.elements:
        for uploaded_file in message.elements:
            file_path = uploaded_file.path  
            
            if file_path.endswith(".csv"):
                csv_path = file_path
            elif file_path.endswith(".pdf"):
                pdf_path = file_path

        if csv_path and pdf_path:
            text2sql = TextToSQL(csv_path, pdf_path, "faiss_index")
            await cl.Message(content="✅ CSV and PDF uploaded successfully! Now ask your question.").send()
        elif csv_path:
            await cl.Message(content="✅ CSV uploaded! Now upload a PDF.").send()
        elif pdf_path:
            await cl.Message(content="✅ PDF uploaded! Now upload a CSV.").send()
        return

    # Ensure both CSV and PDF are uploaded before processing queries
    if not csv_path or not pdf_path:
        await cl.Message(content="⚠️ Please upload both a CSV and a PDF first.").send()
        return

    # Process user query after both files are uploaded
    user_query = message.content.strip()
    
    # Create a placeholder for streaming response
    msg = cl.Message(content="Generating SQL Query...\n")
    await msg.send()
    
    response = text2sql.generate_sql_query(user_query)
    out = json.loads(response)

    # Extract answer and SQL details
    Answer = out.get('Answer', 'No answer provided')
    SQL = out.get('SQL Query', None)
    ans = out.get('SQL Query Answer', None)

    # Stream the SQL answer word by word
    result = f"**Answer:** "
    for word in Answer.split():
        result += word + " "
        msg.content = result
        await msg.update()
        time.sleep(0.1)  # Simulating streaming effect

    # Stream the SQL query word by word
    if SQL:
        result += f"\n\n```sql\n"
        for word in SQL.split():
            result += word + " "
            msg.content = result
            await msg.update()
            time.sleep(0.1)
        result += "\n```"

    # Final update with SQL query output
    msg.content += f"\n\n**Output:** {ans}"
    await msg.update()

    # ✅ **Display the already saved Pie Chart Image**
    file_path = "dynamic_pie_chart.png"
    if os.path.exists(file_path):
        await cl.Message(
            content="📊 **Generated Pie Chart:**",
            elements=[cl.Image(path=file_path)]
        ).send()
    
if __name__ == "__main__":
    cl.run()


In [ ]:
from botocore.credentials import create_assume_role_refresher as carr
from botocore.credentials import DeferredRefreshableCredentials as DRC
from boto3.session import Session
import os

def get_credentials():
    session = Session(region_name="us-east-1")


    session._session._credentials = DRC(
        refresh_using=carr(session.client("sts"),
        {
            "RoleArn": "arn:aws:iam::942286715197:role/app-bedrock-access-900858-us-east-1",
            "RoleSessionName": "test"
        }),
        method="sts-assume-role"
    )

    credentials  = session.get_credentials().get_frozen_credentials()
    access_key = credentials.access_key
    secret_key = credentials.secret_key
    token = credentials.token
    

    # Your AWS credentials
    os.environ["AWS_ACCESS_KEY_ID"] = access_key
    os.environ["AWS_SECRET_ACCESS_KEY"] = secret_key
    os.environ["AWS_SESSION_TOKEN"] = token
    os.environ["AWS_REGION"] = "us-east-1"

    return access_key, secret_key, token




## class to check the credentials update

In [ ]:
import os
from botocore.credentials import create_assume_role_refresher as carr
from botocore.credentials import DeferredRefreshableCredentials as DRC
from boto3.session import Session

class AWSCredentialsManager:
    def __init__(self, role_arn, region="us-east-1"):
        self.role_arn = role_arn
        self.region = region
        self.session = Session(region_name=self.region)
        self.access_key = None
        self.secret_key = None
        self.token = None

    def assume_role(self):
        """Assumes the AWS role and sets credentials."""
        self.session._session._credentials = DRC(
            refresh_using=carr(
                self.session.client("sts"),  
                {
                    "RoleArn": self.role_arn,
                    "RoleSessionName": "test"
                }
            ),
            method="sts-assume-role"
        )

        credentials = self.session.get_credentials().get_frozen_credentials()
        self.access_key = credentials.access_key
        self.secret_key = credentials.secret_key
        self.token = credentials.token

        # Explicitly set the environment variables
        os.environ["AWS_ACCESS_KEY_ID"] = self.access_key
        os.environ["AWS_SECRET_ACCESS_KEY"] = self.secret_key
        os.environ["AWS_SESSION_TOKEN"] = self.token
        os.environ["AWS_REGION"] = self.region

        # Verify they are set
        print("AWS_ACCESS_KEY_ID:", os.environ.get("AWS_ACCESS_KEY_ID"))
        print("AWS_SECRET_ACCESS_KEY:", os.environ.get("AWS_SECRET_ACCESS_KEY"))
        print("AWS_SESSION_TOKEN:", os.environ.get("AWS_SESSION_TOKEN"))
        print("AWS_REGION:", os.environ.get("AWS_REGION"))

        return self.access_key, self.secret_key, self.token

# Usage
aws_manager = AWSCredentialsManager(role_arn="arn:aws:iam::942286715197:role/app-bedrock-access-900858-us-east-1")
aws_manager.assume_role()


In [ ]:
import os
from botocore.credentials import create_assume_role_refresher as carr
from botocore.credentials import DeferredRefreshableCredentials as DRC
from boto3.session import Session

class AWSCredentialsManager:
    def __init__(self, role_arn, region="us-east-1"):
        self.role_arn = role_arn
        self.region = region
        self.session = Session(region_name=self.region)
        self.access_key = None
        self.secret_key = None
        self.token = None

    def assume_role(self):
        """Assumes the AWS role and sets credentials globally."""
        self.session._session._credentials = DRC(
            refresh_using=carr(
                self.session.client("sts"),  
                {
                    "RoleArn": self.role_arn,
                    "RoleSessionName": "test"
                }
            ),
            method="sts-assume-role"
        )

        credentials = self.session.get_credentials().get_frozen_credentials()
        self.access_key = credentials.access_key
        self.secret_key = credentials.secret_key
        self.token = credentials.token

        
        os.system(f'setx AWS_ACCESS_KEY_ID "{self.access_key}"')
        os.system(f'setx AWS_SECRET_ACCESS_KEY "{self.secret_key}"')
        os.system(f'setx AWS_SESSION_TOKEN "{self.token}"')
        os.system(f'setx AWS_REGION "{self.region}"')
        

        # Verify they are set
        print("AWS_ACCESS_KEY_ID:", os.environ.get("AWS_ACCESS_KEY_ID"))
        print("AWS_SECRET_ACCESS_KEY:", os.environ.get("AWS_SECRET_ACCESS_KEY"))
        print("AWS_SESSION_TOKEN:", os.environ.get("AWS_SESSION_TOKEN"))
        print("AWS_REGION:", os.environ.get("AWS_REGION"))

        return self.access_key, self.secret_key, self.token

# Usage
aws_manager = AWSCredentialsManager(role_arn="arn:aws:iam::942286715197:role/app-bedrock-access-900858-us-east-1")
aws_manager.assume_role()


In [ ]:
import os
from botocore.credentials import create_assume_role_refresher as carr
from botocore.credentials import DeferredRefreshableCredentials as DRC
from boto3.session import Session

class AWSCredentialsManager:
    def __init__(self, role_arn, region="us-east-1"):
        self.role_arn = role_arn
        self.region = region
        self.session = Session(region_name=self.region)
        self.access_key = None
        self.secret_key = None
        self.token = None

    def assume_role(self):
        """Assumes the AWS role and sets credentials."""
        self.session._session._credentials = DRC(
            refresh_using=carr(
                self.session.client("sts"),  
                {
                    "RoleArn": self.role_arn,
                    "RoleSessionName": "test"
                }
            ),
            method="sts-assume-role"
        )

        credentials = self.session.get_credentials().get_frozen_credentials()
        self.access_key = credentials.access_key
        self.secret_key = credentials.secret_key
        self.token = credentials.token

        # Explicitly set the environment variables for the current session
        os.environ["AWS_ACCESS_KEY_ID"] = self.access_key
        os.environ["AWS_SECRET_ACCESS_KEY"] = self.secret_key
        os.environ["AWS_SESSION_TOKEN"] = self.token
        os.environ["AWS_REGION"] = self.region

        # Use os.system to set them in the shell as well (for immediate use in the terminal)
        os.system(f'set AWS_ACCESS_KEY_ID={self.access_key}')
        os.system(f'set AWS_SECRET_ACCESS_KEY={self.secret_key}')
        os.system(f'set AWS_SESSION_TOKEN={self.token}')
        os.system(f'set AWS_REGION={self.region}')

        # Verify they are set
        print("AWS_ACCESS_KEY_ID:", os.environ.get("AWS_ACCESS_KEY_ID"))
        print("AWS_SECRET_ACCESS_KEY:", os.environ.get("AWS_SECRET_ACCESS_KEY"))
        print("AWS_SESSION_TOKEN:", os.environ.get("AWS_SESSION_TOKEN"))
        print("AWS_REGION:", os.environ.get("AWS_REGION"))

        return self.access_key, self.secret_key, self.token

# Usage
aws_manager = AWSCredentialsManager(role_arn="arn:aws:iam::942286715197:role/app-bedrock-access-900858-us-east-1")
aws_manager.assume_role()


## AUTOREFRESH CODE

In [ ]:
import os
import configparser
from botocore.credentials import create_assume_role_refresher as carr
from botocore.credentials import DeferredRefreshableCredentials as DRC
from boto3.session import Session

# AWS profiles to update
PROFILES = ["adfd", "test"]

# Role ARN (Modify if needed)
ROLE_ARN = "arn:aws:iam::942286715197:role/app-bedrock-access-900858-us-east-1"

def get_credentials():
    """Fetches temporary AWS credentials using STS assume role."""
    session = Session(region_name="us-east-1")
    session._session._credentials = DRC(
        refresh_using=carr(session.client("sts"), {"RoleArn": ROLE_ARN, "RoleSessionName": "test"}),
        method="sts-assume-role"
    )

    credentials = session.get_credentials().get_frozen_credentials()
    return credentials.access_key, credentials.secret_key, credentials.token

def update_aws_credentials_file(access_key, secret_key, token):
    """Updates ~/.aws/credentials file with new tokens for both profiles."""
    aws_credentials_path = os.path.expanduser("~/.aws/credentials")
    config = configparser.ConfigParser()
    config.read(aws_credentials_path)

    for profile in PROFILES:
        if profile not in config:
            config[profile] = {}

        config[profile]["aws_access_key_id"] = access_key
        config[profile]["aws_secret_access_key"] = secret_key
        config[profile]["aws_session_token"] = token
        config[profile]["region"] = "us-east-1"

    with open(aws_credentials_path, "w") as configfile:
        config.write(configfile)

    print(f"✅ AWS credentials file updated for profiles: {', '.join(PROFILES)}")

def main():
    """Fetches credentials and updates AWS credentials file once (manual execution)."""
    access_key, secret_key, token = get_credentials()
    update_aws_credentials_file(access_key, secret_key, token)

if __name__ == "__main__":
    main()


In [ ]:
import os
import configparser
from dotenv import load_dotenv, set_key
from botocore.credentials import create_assume_role_refresher as carr
from botocore.credentials import DeferredRefreshableCredentials as DRC
from boto3.session import Session

# Load existing environment variables
load_dotenv()

# AWS profiles
PROFILES = ["adfs", "test"]

# Role ARN for `test` profile
ROLE_ARN = "arn:aws:iam::942286715197:role/app-bedrock-access-900858-us-east-1"

# Paths
AWS_CREDENTIALS_PATH = os.path.expanduser("~/.aws/credentials")
ENV_FILE_PATH = os.path.join(os.getcwd(), ".env")  # Assumes .env is in the current directory

def get_credentials():
    """Fetches temporary AWS credentials using STS assume role."""
    session = Session(region_name="us-east-1")
    session._session._credentials = DRC(
        refresh_using=carr(session.client("sts"), {"RoleArn": ROLE_ARN, "RoleSessionName": "test"}),
        method="sts-assume-role"
    )

    credentials = session.get_credentials().get_frozen_credentials()
    return credentials.access_key, credentials.secret_key, credentials.token, ROLE_ARN

def update_aws_credentials_file(access_key, secret_key, token, assumed_role):
    """Updates ~/.aws/credentials file with new tokens for test and syncs adfd's role ARN with '/test' suffix."""
    config = configparser.ConfigParser()
    config.read(AWS_CREDENTIALS_PATH)

    for profile in PROFILES:
        if profile not in config:
            config[profile] = {}

        config[profile]["aws_access_key_id"] = access_key
        config[profile]["aws_secret_access_key"] = secret_key
        config[profile]["aws_session_token"] = token
        config[profile]["region"] = "us-east-1"

    # Sync adfd's assumed role with test and append "/test"
    config["adfs"]["aws_assumed_role_arn"] = assumed_role + "/test"

    with open(AWS_CREDENTIALS_PATH, "w") as configfile:
        config.write(configfile)

    print(f"✅ AWS credentials file updated for profiles: {', '.join(PROFILES)}")
    print(f"🔄 Synced 'adfs' assumed role with 'test': {assumed_role}/test")

def update_env_file(access_key, secret_key, token):
    """Updates .env file with new AWS credentials."""
    if os.path.exists(ENV_FILE_PATH):
        set_key(ENV_FILE_PATH, "AWS_ACCESS_KEY_ID", access_key)
        set_key(ENV_FILE_PATH, "AWS_SECRET_ACCESS_KEY", secret_key)
        set_key(ENV_FILE_PATH, "AWS_SESSION_TOKEN", token)
        print("✅ .env file updated!")
    else:
        print("⚠️ No .env file found! Skipping update.")

def main():
    """Fetches credentials and updates AWS credentials + .env file."""
    access_key, secret_key, token, assumed_role = get_credentials()
    update_aws_credentials_file(access_key, secret_key, token, assumed_role)
    update_env_file(access_key, secret_key, token)

if __name__ == "__main__":
    main()


## ADDED CODE TO GRNERATE THE GRAPH

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import os


client = OpenAI()

def suggest_and_plot(df):
    # Analyze DataFrame structure
    data_summary = df.describe().to_string() + "\n" + str(df.info())

    # Prompt for OpenAI
    prompt = f"""
    Analyze the following DataFrame and suggest the most suitable type of graph:

    DataFrame Summary:
    {data_summary}

    Provide only the graph type (e.g., bar chart, scatter plot, histogram) and the columns to plot.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are a data scientist and graph expert.",
            },
            {
                "role": "user",
                "content": "Task, Goal, or Current Prompt:\n" + prompt,
            },
        ],
    )

    suggestion = response.choices[0].message.content.strip()
    print(f"Suggested Graph: {suggestion}")

    # Auto-generate the plot based on suggestion
    plt.figure(figsize=(10, 6))

    if "bar chart" in suggestion:
        cat_col, num_col = suggestion.split(":")[1].strip().split(",")
        sns.barplot(x=cat_col.strip(), y=num_col.strip(), data=df)
    elif "scatter plot" in suggestion:
        x_col, y_col = suggestion.split(":")[1].strip().split(",")
        sns.scatterplot(x=x_col.strip(), y=y_col.strip(), data=df)
    elif "histogram" in suggestion:
        num_col = suggestion.split(":")[1].strip()
        sns.histplot(df[num_col.strip()], kde=True)
    elif "heatmap" in suggestion:
        sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
    else:
        print("Fallback: Pairplot")
        sns.pairplot(df)

    plt.title("Auto-Generated Graph")
    plt.show()

# Example DataFrame
import pandas as pd
df = pd.read_csv("ElectricCarData_Clean.csv")

# Run the automated graph generator
suggest_and_plot(df)


## FOR CLAUDE

In [ ]:
def generate_graph(self, pdf_context, csv_context, query, json_output_str):
        prompt = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2048,
            "system": """You are an expert AI assistant for Text-to-SQL conversion and data visualization.

            **Task:**
            - Analyze the provided PDF for context and the CSV data structure to understand the schema and answer the user's question:
              1. If it requires data retrieval, generate a SQL query and summarize the answer using the PDF context.
              2. If it doesn't require SQL querying, provide only an explanation and set the SQL query to null.
              3. If the question involves time-based data or any data that would benefit from visualization (e.g., date ranges, timestamps, or categorical distributions), also generate a visualization using matplotlib or seaborn.
            """,

            "messages": [
                {
                    "role": "user",
                    "content": f"""Explanation:\n{pdf_context}\n\nData Structure:\n{csv_context}\n\nQuestion: {query}\n\n**Instructions:**

            Format your response ONLY as a valid JSON object with these fields:
            - Answer: Your explanation based on the PDF context
            - SQL Query: The SQL query to retrieve the requested data, or null if no query is needed
            - SQL Query Answer: The result of the SQL query in a simple format
            - Visualization: When appropriate, include the following for matplotlib or seaborn:
              - chart_type: The type of chart (e.g., 'bar', 'line', 'scatter')
              - x: The x-axis data
              - y: The y-axis data
              - title: The chart title

            For visualizations:
            - Use appropriate chart types based on the data (bar charts for categorical data, line charts for time series, etc.)
            - Keep the specification clean and minimal
            - IMPORTANT: Do not add any text before or after the JSON object. Your entire response should be only the JSON object itself.

            Example of exactly how your response should look:
            {json_output_str}"""
                }
            ],
            "temperature": 0.0
        }

        # Convert prompt to JSON string
        input_text = json.dumps(prompt)

        # Invoke Claude 3.5 Sonnet model
        response = self.client.invoke_model(
            modelId=os.getenv("MODEL_ID"),
            body=input_text
        )

        # Parse the response
        response_body = json.loads(response['body'].read().decode("utf-8"))
        model_response = response_body['content'][0]['text']

        try:
            out = json.loads(model_response)
            if 'Visualization' in out and out['Visualization']:
                self.save_matplotlib_chart(out['Visualization'])
            return out['SQL Query']
        except Exception as e:
            print("Error parsing model output:", e)
            return None

def save_matplotlib_chart(chart_data):
    chart_type = chart_data.get('chart_type', 'bar')
    x = chart_data.get('x', [])
    y = chart_data.get('y', [])
    title = chart_data.get('title', 'Generated Chart')

    plt.figure(figsize=(10, 6))

    if chart_type == 'bar':
        sns.barplot(x=x, y=y)
    elif chart_type == 'line':
        sns.lineplot(x=x, y=y)
    elif chart_type == 'scatter':
        plt.scatter(x, y)
    else:
        print("Unsupported chart type")
        return

    plt.title(title)
    plt.savefig("generated_chart.png")
    print("Chart saved to generated_chart.png")

## TO READ THE INI FILE

In [ ]:
import configparser

# Initialize parser
config = configparser.ConfigParser()

# Read the .ini file
config.read("config.ini")

## chainlit code

In [ ]:
import chainlit as cl
import httpx

FASTAPI_BASE_URL = "http://localhost:7047"  # or your deployed URL

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="👋 Welcome to the Smart Runtime LITE Chainlit assistant!").send()

@cl.on_message
async def on_message(message: cl.Message):
    user_input = message.content.strip().lower()

    if "ping" in user_input:
        async with httpx.AsyncClient() as client:
            response = await client.get(f"{FASTAPI_BASE_URL}/ping")
            await cl.Message(content=f"✅ Ping Response:\n{response.text}").send()

    elif "team" in user_input:
        async with httpx.AsyncClient() as client:
            response = await client.get(f"{FASTAPI_BASE_URL}/team")
            await cl.Message(content=f"🧑‍💻 Team Info:\n{response.text}").send()

    elif "openapi" in user_input:
        async with httpx.AsyncClient() as client:
            response = await client.get(f"{FASTAPI_BASE_URL}/openapi.json")
            await cl.Message(content=f"📘 OpenAPI JSON:\n```json\n{response.text}\n```").send()

    else:
        await cl.Message(content="🤔 I didn't understand that. Try asking about `ping`, `team`, or `openapi`.").send()


In [ ]:
import chainlit as cl
import requests
import uuid

# Static constants
TEAM_ID = ""
TEAM_VERSION = "v1"
HERMES_BASE_URL = f""
AUTH_TOKEN = "<your_token_here>"  # Replace with your actual token

@cl.on_message
def handle_message(message: cl.Message):
    try:
        # Generate dynamic request ID and session ID
        request_id = str(uuid.uuid4())
        session_id = cl.user_session.get("session_id") or str(uuid.uuid4())
        cl.user_session.set("session_id", session_id)

        # You can replace with real username if available
        session_user = cl.user_session.get("user") or "chainlit_user"

        # Build headers dynamically
        headers = {
            "Authorization": f"Bearer {AUTH_TOKEN}",
            "Content-Type": "application/json",
            "request-id": request_id,
            "session-id": session_id,
            "session-user": session_user
        }

        # Payload
        payload = {
            "message": message.content
        }

        # Call Hermes API
        response = requests.post(HERMES_BASE_URL, json=payload, headers=headers)

        if response.status_code == 200:
            data = response.json()
            output = data["task_results"]["messages"][-1]["content"]
            cl.Message(content=f"✅ **Hermes Response:**\n\n{output}").send()
        else:
            cl.Message(content=f"❌ Hermes Error: {response.status_code}\n{response.text}").send()

    except Exception as e:
        cl.Message(content=f"⚠️ Exception: {str(e)}").send()


In [ ]:
import chainlit as cl
import requests

@cl.on_message
async def main(message: cl.Message):
    TEAM_ID = ""
    TEAM_VERSION = ""
    REQUEST_ID = ""
    
    url = f""

    headers = {
        "Content-Type": "application/json",
        "request-id": REQUEST_ID,
        "session-id": "your-session-id",
        "session-user": "your-session-user"
    }

    try:
        response = requests.post(
            url,
            headers=headers,
            json={"message": message.content}
        )

        if response.status_code == 200:
            await cl.Message(content=f"Hermes Response: {response.text}").send()
        else:
            await cl.Message(content=f"Hermes Error {response.status_code}: {response.text}").send()

    except Exception as e:
        await cl.Message(content=f"Error: {str(e)}").send()


## CHAINLIT WITH AUTHENTICATION

In [ ]:
import chainlit as cl
import requests

AUTH_URL = ""
CLIENT_ID = ""
USERNAME = ""
PASSWORD = ""  # Consider storing this securely!
RESOURCE = ""

def get_access_token():
    payload = {
        "grant_type": "password",
        "client_id": CLIENT_ID,
        "username": USERNAME,
        "password": PASSWORD,
        "resource": RESOURCE
    }

    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    
    response = requests.post(AUTH_URL, data=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        raise Exception(f"Token error {response.status_code}: {response.text}")

@cl.on_message
async def main(message: cl.Message):
    try:
        token = get_access_token()

        TEAM_ID = ""
        TEAM_VERSION = ""
        REQUEST_ID = ""
        
        url = f""

        headers = {
            "Content-Type": "application/json",
            "request-id": REQUEST_ID,
            "session-id": "your-session-id",
            "session-user": "your-session-user",
            "Authorization": f"Bearer {token}"  # <-- Add this line
        }

        response = requests.post(
            url,
            headers=headers,
            json={"message": message.content}
        )

        if response.status_code == 200:
            await cl.Message(content=f"Hermes Response: {response.text}").send()
        else:
            await cl.Message(content=f"Hermes Error {response.status_code}: {response.text}").send()

    except Exception as e:
        await cl.Message(content=f"Error: {str(e)}").send()


In [ ]:
import os
import uvicorn
from fastapi import FastAPI
from fastapi.openapi.utils import get_openapi
from starlette.middleware.cors import CORSMiddleware

from app.router.invoke_router import run_invocation_router
from app.router.ping_router import ping_router
from app.router.team_router import team_smart_router
from app.services.json_parser_svc import read_json_str, validate_and_register_team_config
from app.services.log_svc import LogService
from app.utils.agent_utils import current_date_time
from app.utils import config
from app.utils.client.ida_middleware import IDAMiddleware

import chainlit as cl

logger = LogService.get_logger(__name__)

root_path = os.environ.get("APP_ROOT_PATH", config.PROJECT_NAME)

description = """Smart Runtime API"""

app = FastAPI(
    title="Smart CDAO Runtime LITE Application",
    version="0.0.1_" + current_date_time(),
    description=description,
    contact={
        "name": "Smart Runtime LITE",
        "url": "http://go/smartsdk",
        "email": "DG_Agents_Registry_Dev@restricted.chase.com"
    },
    swagger_ui_parameters={
        "filter": True,
        "syntaxHighlight.theme": "nord",
        "defaultModelsExpandDepth": 3,
        "defaultModelExpandDepth": 3,
        "displayRequestDuration": True,
        "defaultModelRendering": "model",
        "persistAuthorization": True,
        "displayOperationId": False,
    }
)

# OpenAPI URL endpoints
@app.get(f"{root_path}/openapi.json", include_in_schema=False)
async def openapi() -> dict:
    return get_openapi(title=app.title, version=app.version, routes=app.routes)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Adjust as needed
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Include routers
app.include_router(ping_router, prefix=root_path)
app.include_router(run_invocation_router, prefix=root_path)
app.include_router(team_smart_router, prefix=root_path)

# Mount the FastAPI app to Chainlit
cl.app.fastapi = app

# Chainlit bot logic (if you want interaction, else remove this part)
@cl.on_message
async def main_logic(message: cl.Message):
    await cl.Message(content=f"You said: {message.content}").send()

# Run uvicorn server inside Chainlit process
if __name__ == "__main__":
    try:
        logger.info("Starting uvicorn Server")
        validate_and_register_team_config(read_json_str())
        uvicorn.run(app, host="0.0.0.0", port=7047, timeout_keep_alive=1800)
        logger.info("Started uvicorn Server")
    except Exception as e:
        logger.error(e)


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))


## OPENSEARCH BLOGS

https://verticalserve.medium.com/advanced-rag-using-aws-opensearch-be7869042405

In [ ]:
df.to_json("data.json")

# Save to JSON file
df.to_json('data.json', orient='records', lines=True)



import json

# Read JSON file
with open('data.json', 'r') as f:
    data = json.load(f)

print(data)

In [ ]:
import json

data = []
with open('data.json', 'r') as f:
    for line in f:
        if line.strip():  # Avoid empty lines
            data.append(json.loads(line))

print(data)

In [ ]:
with open("data.json", "w") as f:
    json.dump(payload, f, indent=4)  # Proper double-quoted JSON


In [ ]:
https://docs.aws.amazon.com/opensearch-service/latest/developerguide/configuration-samples.html

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

# Create AWS authentication
region = 'us-east-1'  # Replace with your region
service = 'es'  # or 'aoss' for serverless
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)

# Initialize OpenSearch client
client = OpenSearch(
    hosts=[{'host': 'your-domain-endpoint.region.es.amazonaws.com', 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

In [ ]:
### GITHUB LINK FOR OPENSEARCH

https://github.com/aws-samples/AI-search-with-amazon-opensearch-service/blob/main/OpenSearchApp/RAG/rag_DocumentSearcher.py

In [ ]:
## API END POINT TO GET THE RESPONSE


import requests

url = "https://your-api-url.com/endpoint"  # Replace with your actual API URL
headers = {
    "Content-Type": "application/json",     # or as per the API docs
    "Authorization": "Bearer YOUR_API_KEY"  # if needed
}
payload = {
    "key1": "value1",
    "key2": "value2"
}

response = requests.post(url, json=payload, headers=headers)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response)  # or response.text if not JSON


In [ ]:
## STREAMLIT CODE 


import streamlit as st
import requests

st.set_page_config(page_title="📡 Single Key API Tester", layout="centered")
st.title("📡 Test Your REST API")

# Input for single key
user_input = st.text_input("Enter input value")

# Send button
if st.button("Send Request"):
    url = "http://localhost:8001/stream/graph/stream"
    headers = {
        "Content-Type": "application/json"
    }

    payload = {
        "key1": user_input
    }

    with st.spinner("Sending request..."):
        try:
            response = requests.post(url, json=payload, headers=headers)
            st.success(f"✅ Status Code: {response.status_code}")
            st.code(response.json(), language="json")
        except Exception as e:
            st.error(f"❌ Request failed: {str(e)}")


In [ ]:
import ast

resp = ast.literal_eval(temp['summary'])


st.markdown(f"{resp['summary']}")

In [ ]:
import pandas as pd

# Read Excel file
df = pd.read_excel("data.xlsx")

# Get unique values from a column (e.g., "Name")
unique_names = df["Name"].dropna().unique().tolist()

# Create dropdown
selected_name = st.selectbox("Choose a name", unique_names)

In [1]:
PROMT = f"""

hey how are you !!


{inp}
"""

NameError: name 'inp' is not defined

In [ ]:
PROMT.foramt()

In [3]:
template = """
hey how are you !!

{first}
{second}
"""

PROMPT = template.format(first="I'm good", second="How about you?")

print(PROMPT)



hey how are you !!

I'm good
How about you?



In [ ]:
# Read Excel file
import pandas as pd
df = pd.read_csv("data.csv")

# Get unique values from a column (e.g., "Name")
unique_names = df["Name"].dropna().unique().tolist()

# Create dropdown
selected_name = st.selectbox("Choose a name", unique_names)

In [ ]:
import pandas as pd

# Assuming the datetime column is called "timestamp"
df["timestamp"] = pd.to_datetime(df["timestamp"], errors='coerce')

# Get unique timestamps formatted as strings
unique_timestamps = df["timestamp"].dropna().dt.strftime("%Y-%m-%d %H:%M:%S").unique().tolist()
unique_timestamps = sorted(unique_timestamps)


In [ ]:
df['eventtimestamp'] = pd.to_datetime(df['eventtimestamp'], unit='s')

ts = df['eventtimestamp'][0] 

ts.strftime("%Y-%m-%d %H:%M:%S") 


In [ ]:
# Convert UNIX timestamp to datetime
df['eventtimestamp'] = pd.to_datetime(df['eventtimestamp'], unit='s')

# Get unique formatted timestamp strings
unique_timestamps = df['eventtimestamp'].dropna().dt.strftime("%Y-%m-%d %H:%M:%S").unique().tolist()

# Sort the list
unique_timestamps = sorted(unique_timestamps)


In [ ]:
st.markdown("<span style='font-weight:bold; font-size:0.8rem;'>ADD YOUR TEXT OVER HER:</span>", unsafe_allow_html=True)


In [ ]:
st.markdown("<h1 style='font-size: 32px;'>Your Custom Title</h1>", unsafe_allow_html=True)


In [ ]:
st.image("your_image_path_or_url.png", width=100)  # Adjust width as needed

In [ ]:
https://www.analyticsvidhya.com/program-dashboard/learner/progress/generative-ai-pinnacle-program

In [ ]:
https://openaipublic.azureedge.net/main/whisper/models/65147644a518d12f04e32d6f3b26facc3f8dd46e5390956a9424a650c0ce22b9/tiny.pt

In [ ]:
https://huggingface.co/ByteDance/LatentSync-1.5/blob/main/latentsync_unet.pt

In [ ]:
import json
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# === Claude 3.5 via Bedrock Config ===
config_list_bedrock = [
    {
        "api_type": "bedrock",
        "model": "meta.llama3-1-70b-instruct-v1:0",
        "aws_region": "us-west-2",
        "aws_access_key": "",
        "aws_secret_key": "",
        "temperature": 0.1,
        "cache_seed": None,  # turn off caching
    }
]

# === Agent 1: Understand ===
understand_agent = AssistantAgent(
    name="CampaignUnderstandAgent",
    system_message=(
        "You are a campaign understanding agent. Interpret marketing goals and extract structured metadata.\n"
        "Return a plan with these keys: 'Target Audience', 'Behavior Filter', 'Preferred Channel', 'Email Tone', 'CTA'."
    ),
    llm_config={"config_list": config_list_bedrock},
)

# === Agent 2: Plan ===
planner_agent = AssistantAgent(
    name="CampaignPlannerAgent",
    system_message=(
        "You are a planning agent. Based on campaign metadata, write an SQL plan using available schema tables:\n"
        "DIGITAL_V, CUSTOMERCORE_V, MARKETING_V. Suggest joins and filters to select eligible users."
    ),
    llm_config={"config_list": config_list_bedrock},
)

# === Agent 3: Execute (mock executor for now) ===
executor_agent = AssistantAgent(
    name="SegmentExecutorAgent",
    system_message=(
        "You are an execution agent. Given an SQL plan, output mock user data as a list of profiles.\n"
        "Each profile includes: name, email, last_login_days, account_type."
    ),
    llm_config={"config_list": config_list_bedrock},
)

# === Agent 4: Generate Emails ===
email_gen_agent = AssistantAgent(
    name="EmailGenAgent",
    system_message=(
        "You are an email generation agent. For each user profile, generate a personalized email.\n"
        "Format: Subject: <...>\nBody: <...>. Use a friendly tone."
    ),
    llm_config={"config_list": config_list_bedrock},
)

# === Agent 5: Review ===
review_agent = AssistantAgent(
    name="EmailReviewerAgent",
    system_message=(
        "You are an email QA reviewer. Review generated emails for tone, spam risk, personalization.\n"
        "Give a brief validation result."
    ),
    llm_config={"config_list": config_list_bedrock},
)

# === User Proxy ===
user_proxy = UserProxyAgent(
    name="UserProxyAgent",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "email_campaign_full", "use_docker": False},
    max_consecutive_auto_reply=1,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "")
)

# === Group Chat ===
agents = [
    user_proxy,
    understand_agent,
    planner_agent,
    executor_agent,
    email_gen_agent,
    review_agent,
]

group = GroupChat(agents=agents, messages=[], max_round=8)
manager = GroupChatManager(groupchat=group, llm_config={"config_list": config_list_bedrock})

# === Start Flow ===
def start_email_campaign():
    campaign_objective = (
        "Launch a campaign to re-engage users who haven't logged in to digital banking in over 45 days.\n"
        "Use email as the primary channel. Tone: friendly and helpful.\n"
        "Offer: highlight new features + easy reactivation.\n"
        "TERMINATE"
    )
    user_proxy.initiate_chat(manager, message=campaign_objective)

    print("\n==== Agent Messages ====")
    all_messages = []
    for msg in group.messages:
        sender = msg.get("name", "Unknown")
        content = msg.get("content", "")
        print(f"\nFrom: {sender}\n{'-'*20}\n{content}\n")
        all_messages.append({"from": sender, "content": content})

    # Save to JSON file
    with open("agent_outputs.json", "w") as f:
        json.dump(all_messages, f, indent=2)

if __name__ == "__main__":
    start_email_campaign()



In [ ]:
import json
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from autogen_ext.models.anthropic import AnthropicBedrockChatCompletionClient, BedrockInfo
from autogen_core.models import ModelInfo

# === Step 1: Setup BedrockInfo ===
bedrock_info = BedrockInfo(
    aws_access_key="YOUR_AWS_ACCESS_KEY",
    aws_secret_key="YOUR_AWS_SECRET_KEY",
    aws_session_token="YOUR_SESSION_TOKEN",  # optional
    aws_region="us-west-2"
)

# === Step 2: Setup Model Client ===
llm_client = AnthropicBedrockChatCompletionClient(
    model="",  # or Claude model ARN if needed
    model_info=ModelInfo(
        vision=False,
        function_calling=True,
        json_output=False,
        family="llama3",
        structured_output=True
    ),
    bedrock_info=bedrock_info
)

# === Step 3: Shared LLM Config ===
llm_config = {
    "client": llm_client
}

# === Agent 1: Understand ===
understand_agent = AssistantAgent(
    name="CampaignUnderstandAgent",
    system_message=(
        "You are a campaign understanding agent. Interpret marketing goals and extract structured metadata.\n"
        "Return a plan with these keys: 'Target Audience', 'Behavior Filter', 'Preferred Channel', 'Email Tone', 'CTA'."
    ),
    llm_config=llm_config,
)

# === Agent 2: Plan ===
planner_agent = AssistantAgent(
    name="CampaignPlannerAgent",
    system_message=(
        "You are a planning agent. Based on campaign metadata, write an SQL plan using available schema tables:\n"
        "DIGITAL_V, CUSTOMERCORE_V, MARKETING_V. Suggest joins and filters to select eligible users."
    ),
    llm_config=llm_config,
)

# === Agent 3: Execute (mock executor) ===
executor_agent = AssistantAgent(
    name="SegmentExecutorAgent",
    system_message=(
        "You are an execution agent. Given an SQL plan, output mock user data as a list of profiles.\n"
        "Each profile includes: name, email, last_login_days, account_type."
    ),
    llm_config=llm_config,
)

# === Agent 4: Generate Emails ===
email_gen_agent = AssistantAgent(
    name="EmailGenAgent",
    system_message=(
        "You are an email generation agent. For each user profile, generate a personalized email.\n"
        "Format: Subject: <...>\nBody: <...>. Use a friendly tone."
    ),
    llm_config=llm_config,
)

# === Agent 5: Review ===
review_agent = AssistantAgent(
    name="EmailReviewerAgent",
    system_message=(
        "You are an email QA reviewer. Review generated emails for tone, spam risk, personalization.\n"
        "Give a brief validation result."
    ),
    llm_config=llm_config,
)

# === User Proxy ===
user_proxy = UserProxyAgent(
    name="UserProxyAgent",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "email_campaign_full", "use_docker": False},
    max_consecutive_auto_reply=1,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "")
)

# === Group Chat ===
agents = [
    user_proxy,
    understand_agent,
    planner_agent,
    executor_agent,
    email_gen_agent,
    review_agent,
]

group = GroupChat(agents=agents, messages=[], max_round=8)
manager = GroupChatManager(groupchat=group, llm_config=llm_config)

# === Main Execution ===
def start_email_campaign():
    campaign_objective = (
        "Launch a campaign to re-engage users who haven't logged in to digital banking in over 45 days.\n"
        "Use email as the primary channel. Tone: friendly and helpful.\n"
        "Offer: highlight new features + easy reactivation.\n"
        "TERMINATE"
    )
    user_proxy.initiate_chat(manager, message=campaign_objective)

    print("\n==== Agent Messages ====")
    all_messages = []
    for msg in group.messages:
        sender = msg.get("name", "Unknown")
        content = msg.get("content", "")
        print(f"\nFrom: {sender}\n{'-'*20}\n{content}\n")
        all_messages.append({"from": sender, "content": content})

    # Save to JSON
    with open("agent_outputs.json", "w") as f:
        json.dump(all_messages, f, indent=2)

if __name__ == "__main__":
    start_email_campaign()


In [ ]:
from autogen import AssistantAgent
from autogen_ext.models.anthropic import AnthropicBedrockChatCompletionClient, BedrockInfo
from autogen_core.models import ModelInfo

# === Step 1: Setup Bedrock Credentials ===
bedrock_info = BedrockInfo(
    aws_access_key="YOUR_ACCESS_KEY",         # 🟡 Replace this
    aws_secret_key="YOUR_SECRET_KEY",         # 🟡 Replace this
    aws_region="us-west-2"
)

# === Step 2: Initialize Claude Client via Bedrock ===
llm_client = AnthropicBedrockChatCompletionClient(
    model="anthropic.claude-3-sonnet-20240229-v1:0",  # ✅ Claude 3.5 Sonnet ARN (or another valid Bedrock model)
    model_info=ModelInfo(
        vision=False,
        function_calling=True,
        json_output=False,
        family="claude-3",
        structured_output=True
    ),
    bedrock_info=bedrock_info
)

# === Step 3: Agent 1 - Campaign Understand Agent ===
understand_agent = AssistantAgent(
    name="CampaignUnderstandAgent",
    system_message=(
        "You are a campaign understanding agent. Interpret marketing goals and extract structured metadata.\n"
        "Return a plan with these keys: 'Target Audience', 'Behavior Filter', 'Preferred Channel', 'Email Tone', 'CTA'."
    ),
    llm_config={"client": llm_client}  # ✅ Use Claude client
)

# === Step 4: Test Run ===
response = understand_agent.generate_reply(messages=[
    {
        "role": "user",
        "content": "We want to re-engage digital banking users inactive for over 45 days. Use friendly tone and email channel."
    }
])

# === Output Result ===
print("🧠 Agent 1 Response:\n", response)


In [ ]:
import asyncio
from autogen import AssistantAgent
from autogen_ext.models.anthropic import AnthropicBedrockChatCompletionClient
from autogen_core.models import ModelInfo

# === Step 1: Setup Bedrock Credentials ===
# Option A: Use AWS credentials from environment/profile (recommended)
# Make sure you have AWS CLI configured or set these environment variables:
# export AWS_ACCESS_KEY_ID="your_access_key"
# export AWS_SECRET_ACCESS_KEY="your_secret_key" 
# export AWS_DEFAULT_REGION="us-west-2"

# Option B: Explicit credentials (not recommended for production)
bedrock_config = {
    "aws_access_key_id": "YOUR_ACCESS_KEY",      # Replace with actual key
    "aws_secret_access_key": "YOUR_SECRET_KEY",  # Replace with actual secret
    "region_name": "us-west-2"                   # Ensure Claude is available in this region
}

# === Step 2: Initialize Claude Client via Bedrock ===
llm_client = AnthropicBedrockChatCompletionClient(
    # ✅ Use correct Claude 3.5 Sonnet model ID for Bedrock
    model="model arn ",  # Updated model ID
    
    # ✅ Correct model info configuration
    model_info=ModelInfo(
        vision=True,           # Claude 3.5 Sonnet supports vision
        function_calling=True, # Supports function calling
        json_output=True,      # Supports JSON output
        family="claude-3.5"    # Correct family name
    ),
    
    # ✅ Pass credentials properly
    **bedrock_config  # Unpack the credentials
)

# === Step 3: Agent 1 - Campaign Understand Agent ===
understand_agent = AssistantAgent(
    name="CampaignUnderstandAgent",
    system_message=(
        "You are a campaign understanding agent. Interpret marketing goals and extract structured metadata.\n"
        "Return a JSON plan with these keys: 'target_audience', 'behavior_filter', 'preferred_channel', 'email_tone', 'cta'.\n"
        "Be specific and actionable in your recommendations."
    ),
    llm_config={
        "config_list": [{
            "model": "anthropic.claude-3-5-sonnet-20241022-v2:0",
            "api_type": "bedrock",
            "client": llm_client
        }],
        "temperature": 0.7,
        "max_tokens": 1000
    }
)

# === Step 4: Async Test Run (Required for newer AutoGen versions) ===
async def test_agent():
    try:
        response = await understand_agent.a_generate_reply(messages=[
            {
                "role": "user", 
                "content": "We want to re-engage digital banking users inactive for over 45 days. Use friendly tone and email channel."
            }
        ])
        return response
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# === Step 5: Alternative Synchronous Approach ===
def test_agent_sync():
    try:
        # For synchronous call, ensure proper message format
        messages = [
            {
                "role": "user",
                "content": "We want to re-engage digital banking users inactive for over 45 days. Use friendly tone and email channel."
            }
        ]
        
        response = understand_agent.generate_reply(
            messages=messages,
            sender=None  # Add sender parameter
        )
        return response
        
    except Exception as e:
        print(f"❌ Sync Error: {e}")
        print(f"Error type: {type(e)}")
        return None

# === Step 6: Run the test ===
if __name__ == "__main__":
    print("🚀 Testing AutoGen with Claude via Bedrock...")
    
    # Try async first
    try:
        result = asyncio.run(test_agent())
        if result:
            print("🧠 Async Agent Response:\n", result)
    except Exception as e:
        print(f"Async failed: {e}")
        
        # Fallback to sync
        print("\n🔄 Trying synchronous approach...")
        result = test_agent_sync()
        if result:
            print("🧠 Sync Agent Response:\n", result)



In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio

if __name__ == "__main__":
    print("🚀 Testing AutoGen with Claude via Bedrock...")
    
    try:
        result = asyncio.run(test_agent())
        if result:
            print("🧠 Async Agent Response:\n", result)
    except Exception as e:
        print(f"Async failed: {e}")
        
        # Fallback to sync
        print("\n🔄 Trying synchronous approach...")
        result = test_agent_sync()
        if result:
            print("🧠 Sync Agent Response:\n", result)


In [1]:
import pandas as pd

# Sample row (simulated using a dictionary or named tuple)
row = dict(sarevsh='ajsja', hsjas='asjha', aksjia='alsnj')

# Convert to DataFrame
df = pd.DataFrame([row])

print(df)


  sarevsh  hsjas aksjia
0   ajsja  asjha  alsnj
